# JTVAE embedding
This is a molecule embedding using the JunctionTree VAE, as implemented in DGLLifeSci.
It doesn't yet work perfectly, since the vocabulary of the model pretrained on ZINC and the vocabulary of LINCS & Trapnell doesn't always match (leading to encoding errors).

The errors may be fixed by retraining, but this is good enough for now (at least on Trapnell).

Number of errors: 7/189(Trapnell), 4548/17000(LINCS).

In [1]:
import pickle
from pathlib import Path

import pandas as pd
import rdkit
import torch
from dgllife.data import JTVAEDataset, JTVAECollator
from dgllife.model import load_pretrained
from tqdm import tqdm

print(rdkit.__version__)
print(torch.__version__)
assert torch.cuda.is_available()

[10:44:33] Using backend: pytorch
/opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/icb/simon.boehm/miniconda3/envs/jtvae_dgl/lib/python3.7/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.1.so: cannot open shared object file: No such file or directory


2018.09.3
1.10.1


In [6]:
from dgllife.utils import JTVAEVocab
from dgllife.model import JTNNVAE

from_pretrained = False
if from_pretrained:
    model = load_pretrained("JTVAE_ZINC_no_kl")
else:
    trainfile = "data/train2c12dbbb06a7f8cb4055bb030a27e624.txt"
    modelfile = "pre_model_all/model.epoch-0-iter-299"
    vocab = JTVAEVocab(file_path=trainfile)

    save_path = "vocab_jupyter.pkl"
    with open(save_path, "wb") as f:
        pickle.dump(vocab, f)
    model = JTNNVAE(vocab=vocab,
                    hidden_size=450,
                    latent_size=56,
                    depth=3)
    model.load_state_dict(torch.load(modelfile, map_location="cpu"))


In [7]:
model = model.to("cuda")

I tried keklulization, but it didn't help

In [8]:
def canonicalize(smiles: str):
    return smiles

In [4]:
x = pd.read_csv("../lincs_trapnell.smiles")

In [5]:
x.to_csv("kekuleSmiles.smiles", index=False, header=None)

In [6]:
dataset = JTVAEDataset("kekuleSmiles.smiles", vocab=model.vocab, training=False)
collator = JTVAECollator(training=False)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collator, drop_last=True)

NameError: name 'model' is not defined

In [ ]:
get_data = lambda idx: collator([dataset[idx]])
errors = []
smiles = []
latents = []
for i in tqdm(range(len(dataset))):
    try:
        _, batch_tree_graphs, batch_mol_graphs = get_data(i)
        batch_tree_graphs = batch_tree_graphs.to("cuda")
        batch_mol_graphs = batch_mol_graphs.to("cuda")
        with torch.no_grad():
            _, tree_vec, mol_vec = model.encode(batch_tree_graphs, batch_mol_graphs)
        latent = torch.cat([model.T_mean(tree_vec), model.G_mean(mol_vec)], dim=1)
        latents.append(latent)
        smiles.append(dataset.data[i])
    except Exception as e:
        errors.append((dataset.data[i], e))

  0%|                                                                           | 0/17833 [00:00<?, ?it/s]/home/icb/simon.boehm/miniconda3/envs/jtvae_dgl/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  return warnings.warn(message, category=category, stacklevel=1)
  6%|███▌                                                          | 1014/17833 [03:18<1:01:51,  4.53it/s]

In [ ]:
len(errors)

In [ ]:
errors

In [11]:
assert len(latents) == len(smiles)

In [17]:
np_latents = [latent.squeeze().cpu().detach().numpy() for latent in latents]
final_df = pd.DataFrame(np_latents, index=smiles,
                        columns=[f"latent_{i + 1}" for i in range(np_latents[0].shape[0])])
final_df.to_parquet("data/jtvae_dgl.parquet")

In [15]:
final_df

,latent_1,latent_2,latent_3,latent_4,latent_5,latent_6,latent_7,latent_8,latent_9,latent_10,...,latent_47,latent_48,latent_49,latent_50,latent_51,latent_52,latent_53,latent_54,latent_55,latent_56
C[C@H](NC(=O)/C(C#N)=C/c1cccc(Br)n1)c1ccccc1,-2.951031,-5.289873,-12.151762,6.205541,7.253455,12.337790,-12.547594,9.394004,2.130386,10.193287,...,0.555162,-0.337887,0.359012,-0.029846,-4.738989,0.279441,0.116574,0.945011,-0.343293,-0.473597
Cc1cc(Nc2cc(CN3CCOCC3)c3nc(C)c(Cc4ccc(Cl)cc4F)n3n2)[nH]n1,2.817028,-14.199235,2.602090,-1.812574,-10.057611,-1.675966,-15.421948,25.793367,-1.076326,-7.611788,...,-0.314179,-0.223788,-0.730887,-0.533293,-2.818021,-0.248069,-0.539395,0.287021,-0.080546,-0.176129
Cc1cc(N2CCOCC2)cc2[nH]c(-c3c(NCC(O)c4cccc(Cl)c4)cc[nH]c3=O)nc12,5.354123,-12.014553,6.532739,13.496149,0.224052,7.808585,-7.222305,16.805893,-23.045683,-1.204091,...,0.125229,0.021564,-0.490427,-0.158655,-0.303421,0.060854,0.254467,-0.011696,0.133957,0.133336
Cl.Cl.c1ccc([C@@H]2C[C@H]2NC2CCNCC2)cc1,6.619146,-4.298629,14.509826,11.098990,6.473425,-4.572600,-3.073167,6.496140,5.623781,-1.876796,...,-0.135496,0.120736,-0.330616,0.106106,-2.455133,-0.195220,-0.339859,0.141549,0.491087,0.261954
O=C(c1ccc(/C=C/c2n[nH]c3ccccc23)cc1)N1CCNCC1,-1.737819,4.922736,0.329061,7.835114,15.684121,1.067316,7.796133,4.631086,2.695632,2.234314,...,0.361858,0.154207,0.424318,-0.102719,-4.824892,0.304993,-0.067202,0.908301,-0.060837,0.199201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CC1(C)C=C2C(=[N+]([O-])c3cc4c(cc32)C=CC(C)(C)O4)C(C)(C)C1,1.611027,11.635976,-6.135553,-5.009007,3.190311,-6.632552,-6.322329,13.814123,-7.321885,8.943810,...,-0.528293,-0.239322,-0.152673,-0.779070,-2.366998,-0.177732,-0.352130,0.465450,-0.259728,0.330294
C[C@@H]1CC(=O)NN=C1c1ccc(Cl)cc1,0.483706,1.880215,7.449620,1.906647,8.257718,-1.502681,-9.334446,12.634336,3.009709,-0.243049,...,-0.293528,-0.295376,0.138144,0.124643,-4.890343,0.153882,0.124399,0.257779,-0.040345,0.181931
Cc1cc(CS(=O)(=O)c2ccccc2)cc(OCc2ccc(CN3CCC[C@@H]3CO)cc2)c1,0.508416,-1.055624,5.894164,8.405050,6.109705,2.316949,10.134815,22.346865,-23.848000,-9.663993,...,0.055109,0.196629,0.072201,0.064545,-3.252289,0.306187,0.191910,0.652843,-0.242031,-0.082915
CN(C)CCOc1ccc(/C(=C(\CCCl)c2ccccc2)c2ccccc2)cc1,0.345383,-5.407078,-10.009950,12.500641,-3.964427,3.735437,-4.372162,14.781828,-17.718584,-20.696524,...,-0.070567,0.555675,0.464044,-0.091776,-7.976848,0.163267,-0.064775,1.394701,-0.057060,-0.181299
